<a href="https://colab.research.google.com/github/lennart194/thesis-code/blob/main/RF_different_costs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install ddop

In [ ]:
import pandas as pd
import numpy as np

from ddop.newsvendor import SampleAverageApproximationNewsvendor
from ddop.newsvendor import RandomForestWeightedNewsvendor

from sklearn.model_selection import RandomizedSearchCV

from ddop.metrics import average_costs
from ddop.metrics import prescriptiveness_score

from ddop.metrics import make_scorer

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/M5/ultimative_set.csv')
data = data.drop(columns=['Unnamed: 0'])
data = data.sort_values(by=['item_id', 'date'])
data = data.set_index(['item_id', 'date'])

In [ ]:
X = data.drop(columns=['demand'])
Y = data['demand']

In [ ]:
X_train = X.loc[(slice(None), slice(None, "2015")), :]
X_test = X.loc[(slice(None), slice("2015", '2017')), :]

Y_train = Y.loc[(slice(None), slice(None, "2015"))]
Y_test = Y.loc[(slice(None), slice("2015", "2017"))]

In [ ]:
max_depth = np.arange(7, 24)
n_estimators = np.arange(65, 101)
min_samples_leaf = np.arange(2, 6)
param_random = dict(max_depth = max_depth, n_estimators = n_estimators, min_samples_leaf = min_samples_leaf)

scorer_avc = make_scorer(average_costs, greater_is_better=False)

In [ ]:
prods = list(range(0,25))

In [ ]:
best_param_maxd_90 = []
best_param_nest_90 = []
best_param_minsl_90 = []

for prod in prods:

  cu = round(X_train['sell_price'].loc[prod].mean(),2)
  co_90 = round((cu-(0.90*cu))/0.90,2)
  
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_90.append(maxd)
  best_param_nest_90.append(nest)
  best_param_minsl_90.append(minsl)

In [ ]:
result_rf_90_pscr = []
for prod in prods:
  
  cu = round(X_train['sell_price'].loc[prod].mean(),2)
  co_90 = round((cu-(0.90*cu))/0.90,2)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_90, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_90[prod], n_estimators = best_param_nest_90[prod],
                                       min_samples_leaf = best_param_minsl_90[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_90)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_90)
  result_rf_90_pscr.append(pscr)

In [ ]:
RF_separate_pscr_90 = pd.DataFrame(result_rf_90_pscr, columns=['RF_pscr_90'])
RF_separate_pscr_90.index.name = 'item_id'

In [ ]:
best_param_maxd_75 = []
best_param_nest_75 = []
best_param_minsl_75 = []

for prod in prods:

  cu = round(X_train['sell_price'].loc[prod].mean(),2)
  co_75 = round((cu-(0.75*cu))/0.75,2)
  
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_75.append(maxd)
  best_param_nest_75.append(nest)
  best_param_minsl_75.append(minsl)

In [ ]:
result_rf_75_pscr = []
for prod in prods:

  cu = round(X_train['sell_price'].loc[prod].mean(),2)
  co_75 = round((cu-(0.75*cu))/0.75,2)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_75, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_75[prod], n_estimators = best_param_nest_75[prod],
                                       min_samples_leaf = best_param_minsl_75[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_75)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_75)
  result_rf_75_pscr.append(pscr)

In [ ]:
RF_separate_pscr_75 = pd.DataFrame(result_rf_75_pscr, columns=['RF_pscr_75'])
RF_separate_pscr_75.index.name = 'item_id'

In [ ]:
best_param_maxd_50 = []
best_param_nest_50 = []
best_param_minsl_50 = []

for prod in prods:

  cu = round(X_train['sell_price'].loc[prod].mean(),2)
  co_50 = round((cu-(0.50*cu))/0.50,2)
  
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse')

  random = RandomizedSearchCV(rf, param_random, cv=5, scoring = scorer_avc)
  random.fit(X_train.loc[prod], Y_train.loc[prod])
    
  maxd = random.best_params_.get('max_depth')
  nest = random.best_params_.get('n_estimators')
  minsl = random.best_params_.get('min_samples_leaf')

  best_param_maxd_50.append(maxd)
  best_param_nest_50.append(nest)
  best_param_minsl_50.append(minsl)

In [ ]:
result_rf_50_pscr = []
for prod in prods:

  cu = round(X_train['sell_price'].loc[prod].mean(),2)
  co_50 = round((cu-(0.50*cu))/0.50,2)
  
  rf = RandomForestWeightedNewsvendor(cu = cu, co = co_50, random_state = 42, criterion = 'mse',
                                       max_depth = best_param_maxd_50[prod], n_estimators = best_param_nest_50[prod],
                                       min_samples_leaf = best_param_minsl_50[prod])
  rf.fit(X_train.loc[prod], Y_train.loc[prod])
  preds_rf = rf.predict(X_test.loc[prod])


  saa = SampleAverageApproximationNewsvendor(cu = cu, co = co_50)
  saa.fit(Y_train.loc[prod])
  preds_saa = saa.predict(n_steps = 508)
  
  pscr = prescriptiveness_score(Y_test.loc[prod], preds_rf, preds_saa, cu = cu, co = co_50)
  result_rf_50_pscr.append(pscr)

In [ ]:
RF_separate_pscr_50 = pd.DataFrame(result_rf_50_pscr, columns=['RF_pscr_50'])
RF_separate_pscr_50.index.name = 'item_id'

In [ ]:
RF_separated_pscr = pd.concat([RF_separate_pscr_90, RF_separate_pscr_75, RF_separate_pscr_50], axis=1)
RF_separated_pscr

,RF_pscr_90,RF_pscr_75,RF_pscr_50
item_id,,,
0,0.399319,0.317825,0.307923
1,0.519607,0.480878,0.440160
2,0.212054,0.138039,0.135685
3,0.674064,0.638255,0.543425
4,0.357625,0.266043,0.221009
5,0.408004,0.381765,0.244444
6,0.459077,0.455709,0.418310
7,0.413813,0.258656,0.191323
8,0.388507,0.311825,0.254415


In [ ]:
RF_separated_pscr.to_csv('/content/drive/MyDrive/M5/RF_separated_costs_pscr.csv')